In [27]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds
from datetime import datetime

import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [28]:
df_ratings = pd.read_csv('data/ml-latest-small/ratings.csv')
df_movies = pd.read_csv('data/ml-latest-small/movies.csv')

## Matrix Factorization using SVD
with Feature scaling using Standardization

 Standardization이 (X - X의 평균값) / (X의 표준편차) 이렇게 하는건데 여기서는 표준편차를 안 나눴네..? 나중에 해보자

In [29]:
def MatrixFactorizaion(df_ratings, df_movies):
    df_ratings = pd.read_csv('data/ml-latest-small/ratings.csv')
    df_movies = pd.read_csv('data/ml-latest-small/movies.csv')

    df_ratings=df_ratings[:2000000]
    df_movie_features = df_ratings.pivot(
        index='userId',
        columns='movieId',
        values='rating'
    ).fillna(0)
    
    ## Data normalizaiion ##
    # pivot table을 matrix로 변환
    R = df_movie_features.as_matrix()
    # 각 user들의 rating 평균을 구함
    user_ratings_mean = np.mean(R, axis = 1)
    # R_demeaned : 사용자-영화에 대해 사용자 평균 평점을 뺀 것.
    R_demeaned = R - user_ratings_mean.reshape(-1, 1)
    
    M_demeaned = pd.DataFrame(R_demeaned, columns = df_movie_features.columns)
    
    
    # U 행렬, sigma 행렬, V 전치 행렬을 반환.
    U, sigma, Vt = svds(M_demeaned, k = 50)
    # 0이 포함된 대칭행렬로 변환
    sigma = np.diag(sigma)
    
    # U, Sigma, Vt의 내적을 수행하면, 다시 원본 행렬로 복원 + 사용자 평균 rating을 적용 
    svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
    all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
    
    preds_df = pd.DataFrame(all_user_predicted_ratings, columns = df_movie_features.columns)
    
    return preds_df

In [30]:
preds_df = MatrixFactorizaion(df_ratings, df_movies)
preds_df.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
0,-0.054239,0.045130,-0.004835,-0.019817,-0.011284,0.041373,-0.007822,-0.017188,0.012246,0.037670,...,-0.005258,-0.005453,0.012369,-0.004991,-0.004639,-0.019055,0.021402,-0.006365,-0.006098,-0.004819
1,0.419835,1.406440,-0.188807,0.156658,0.268032,0.414698,0.052172,0.044728,-0.020198,2.220256,...,-0.005909,-0.003974,-0.012555,-0.003555,-0.002711,-0.071621,-0.016212,0.001047,-0.001468,-0.006577
2,1.345619,0.266505,-0.011962,0.012278,0.079508,0.090960,-0.122094,0.031327,-0.018023,0.141176,...,-0.002647,-0.002364,-0.010153,0.000277,-0.000116,-0.018063,-0.015761,0.010611,0.006792,-0.006357
3,1.133455,1.046982,0.141275,0.081841,-0.339675,-1.484659,-0.263096,-0.169750,-0.021862,1.611664,...,0.020805,0.000410,0.056040,-0.002817,-0.000767,0.159159,0.087519,-0.030854,-0.021279,0.048529
4,1.389578,1.466495,0.605557,-0.029647,0.729380,-0.118539,-0.026017,0.065577,-0.156655,0.307926,...,-0.007422,-0.011810,0.006644,-0.005159,-0.001249,-0.034658,0.016456,0.001710,-0.004166,-0.001864


## All user prediction system

In [31]:
def AllUserPrediction(preds_df, userID, df_movies, df_ratings, num_recommendations=5):
    df_ratings = pd.read_csv('data/ml-latest-small/ratings.csv')
    df_movies = pd.read_csv('data/ml-latest-small/movies.csv')

    user_row_number = userID - 1 # UserID starts at 1, not 0
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 영화 데이터 정렬 -> 영화 평점이 높은 순으로 정렬
    sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False) # UserID starts at 1
    
    # 원본 평점 데이터에서 userId에 해당하는 데이터를 뽑아냄
    user_data = df_ratings[df_ratings.userId == (userID)]
    
    # 위에서 뽑은 user_data와 원본 영화 데이터를 합침
    user_full = (user_data.merge(df_movies, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False))
    user_full = user_data.merge(df_movies, on = 'movieId').sort_values(['rating'], ascending=False)
    
    # 원본 영화 데이터에서 사용자가 본 영화 데이터를 제외한 데이터를 추출
    recommendations = df_movies[~df_movies['movieId'].isin(user_full['movieId'])]
    # 사용자의 영화 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = recommendations.merge( pd.DataFrame(sorted_user_predictions).reset_index(), on = 'movieId')
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]

    return recommendations

#### 330번 user에게 추천해줄 movie list

In [32]:
predictions = AllUserPrediction(preds_df, 330, df_movies, df_ratings, 10)
predictions.head()

,movieId,title,genres,Predictions
246,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,2.190762
487,590,Dances with Wolves (1990),Adventure|Drama|Western,1.924099
294,356,Forrest Gump (1994),Comedy|Drama|Romance|War,1.732990
40,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1.579084
316,380,True Lies (1994),Action|Adventure|Comedy|Romance|Thriller,1.345957


## Generate 'user - time(6hour term)' Matrix 

In [33]:
def UserTimeMatrix(df_ratings):
    df_ratings = pd.read_csv('data/ml-latest-small/ratings.csv')
    
    # original dataframe에서 moiveId, rating, drop,
    rd = df_ratings
    rd.drop('movieId', axis = 1, inplace = True)
    rd.drop('rating', axis = 1, inplace = True)
    # timestamp2datetime
    rd['Hour_6']=[datetime.fromtimestamp(x).strftime('%H') for x in rd['timestamp']]
    rd.drop('timestamp', axis = 1, inplace = True)
    rd = rd.reindex(columns=['Hour_6','userId'])
    # datetime 기준으로 정렬
    rd = rd.sort_values(by='Hour_6')
    
    # datetime2Hour_6term
    rd.loc[(rd['Hour_6']=="00") | (rd["Hour_6"]=="01") | 
           (rd["Hour_6"]=="02") | (rd["Hour_6"]=="03"),"Hour_6"] = "h1"
    rd.loc[(rd['Hour_6']=="04") | (rd["Hour_6"]=="05") | 
           (rd["Hour_6"]=="06") | (rd["Hour_6"]=="07"),"Hour_6"] = "h2"
    rd.loc[(rd['Hour_6']=="08") | (rd["Hour_6"]=="09") | 
           (rd["Hour_6"]=="10") | (rd["Hour_6"]=="11"),"Hour_6"] = "h3"
    rd.loc[(rd['Hour_6']=="12") | (rd["Hour_6"]=="13") | 
           (rd["Hour_6"]=="14") | (rd["Hour_6"]=="15"),"Hour_6"] = "h4"
    rd.loc[(rd['Hour_6']=="16") | (rd["Hour_6"]=="17") | 
           (rd["Hour_6"]=="18") | (rd["Hour_6"]=="19"),"Hour_6"] = "h5"
    rd.loc[(rd['Hour_6']=="20") | (rd["Hour_6"]=="21") | 
           (rd["Hour_6"]=="22") | (rd["Hour_6"]=="23"),"Hour_6"] = "h6"
    
    return rd

In [34]:
M1 = UserTimeMatrix(df_ratings)
M1.head()

,Hour_6,userId
100003,h1,671
43614,h1,311
85514,h1,574
85513,h1,574
85512,h1,574


## Generate 'user - prediction(top 5)' Matrix

In [39]:
def UserPredictionMatrix(predictions):
    user_title = {}
    for i in range(1, 672):
        predictions = AllUserPrediction(preds_df, i, df_movies, df_ratings, 10)
        df = predictions[:5]
        title = df[['title']]
        #title.drop(index='userId')
        title.reset_index(drop=True)
        title.rename(columns = {"title": i}, inplace = True)
        dict_from_df = title.to_dict('list')
        user_title.update(dict_from_df)
        df = pd.DataFrame(user_title)
        df = pd.DataFrame(list(user_title.items()),columns =['userId','title'])
    
    return df

In [40]:
M2 = UserPredictionMatrix(predictions)
M2.head()

,userId,title
0,1,"[Star Trek II: The Wrath of Khan (1982), Rocky (1976), Who Framed Roger Rabbit? (1988), Galaxy Quest (1999), Ladyhawke (1985)]"
1,2,"[True Lies (1994), Shawshank Redemption, The (1994), Beauty and the Beast (1991), Stargate (1994), Ace Ventura: Pet Detective (1994)]"
2,3,"[Matrix, The (1999), Lord of the Rings: The Fellowship of the Ring, The (2001), Lord of the Rings: The Two Towers, The (2002), Star Wars: Episode IV - A New Hope (1977), Toy Story (1995)]"
3,4,"[Beauty and the Beast (1991), Raising Arizona (1987), Glory (1989), Platoon (1986), Monty Python's Life of Brian (1979)]"
4,5,"[Men in Black (a.k.a. MIB) (1997), Sleepless in Seattle (1993), Incredibles, The (2004), Catch Me If You Can (2002), Groundhog Day (1993)]"


-----

### 00 ~ 04시간에 자주 시청하는 유저들 중  "Toy Story (1995)" 와 비슷한 취향을 가지고 있어(predict) 초대장을 보낼 유저들

In [60]:
def main(M1,M2):
    # Hour_6 변수 선언
    h1 = M1.loc[M1["Hour_6"] == "h1", :]  # h1 : 00 ~ 03
    h2 = M1.loc[M1["Hour_6"] == "h2", :]  # h2 : 04 ~ 07
    h3 = M1.loc[M1["Hour_6"] == "h3", :]  # h3 : 08 ~ 11
    h4 = M1.loc[M1["Hour_6"] == "h4", :]  # h4 : 12 ~ 15
    h5 = M1.loc[M1["Hour_6"] == "h5", :]  # h5 : 16  ~19
    h6 = M1.loc[M1["Hour_6"] == "h6", :]  # h6 : 20 ~ 23
    
    h1 = pd.merge(h1, M2, on="userId")
    h1 = h1.drop_duplicates(["userId"])
    print(h1)
    
    lst = ["Toy Story (1995)"] # filter list
    m = h1["title"].apply(lambda s: len(set(s) & set(lst)) > 0)
    
    return h1[m]

In [61]:
pd.set_option('display.max.colwidth', 200)
main(M1,M2)

      Hour_6  userId  \
0         h1     671   
25        h1     311   
178       h1     574   
425       h1     299   
714       h1     382   
771       h1     111   
1069      h1     612   
1135      h1     529   
1440      h1      22   
1452      h1     380   
1753      h1     213   
2663      h1      23   
2864      h1     502   
2917      h1     173   
2919      h1     369   
2970      h1     580   
3226      h1     105   
3295      h1     225   
3323      h1     300   
3330      h1     607   
3616      h1     177   
3641      h1     509   
3814      h1      24   
3825      h1     176   
3833      h1      61   
3998      h1      86   
4188      h1     624   
4602      h1     569   
4652      h1     384   
4943      h1     295   
...      ...     ...   
18006     h1     292   
18007     h1     488   
18038     h1     471   
18041     h1     571   
18087     h1      87   
18118     h1     572   
18184     h1     339   
18229     h1     336   
18260     h1     189   
18372     h1    

,Hour_6,userId,title
3833,h1,61,"[Toy Story (1995), Good Will Hunting (1997), Eternal Sunshine of the Spotless Mind (2004), Truman Show, The (1998), Apollo 13 (1995)]"
4943,h1,295,"[Toy Story (1995), Groundhog Day (1993), Twelve Monkeys (a.k.a. 12 Monkeys) (1995), Ghost (1990), Aliens (1986)]"
11014,h1,427,"[Eternal Sunshine of the Spotless Mind (2004), Pianist, The (2002), Royal Tenenbaums, The (2001), High Fidelity (2000), Toy Story (1995)]"
11404,h1,659,"[Toy Story (1995), Babe (1995), Beauty and the Beast (1991), Clear and Present Danger (1994), Interview with the Vampire: The Vampire Chronicles (1994)]"
11734,h1,550,"[Toy Story (1995), Star Wars: Episode IV - A New Hope (1977), Back to the Future (1985), Blade Runner (1982), Usual Suspects, The (1995)]"
11975,h1,394,"[Lord of the Rings: The Return of the King, The (2003), Lord of the Rings: The Two Towers, The (2002), Toy Story (1995), American Beauty (1999), Jurassic Park (1993)]"
14581,h1,234,"[Back to the Future (1985), Lion King, The (1994), Eternal Sunshine of the Spotless Mind (2004), Toy Story (1995), Dark Knight, The (2008)]"
15338,h1,190,"[Fight Club (1999), Blair Witch Project, The (1999), Toy Story (1995), Bug's Life, A (1998), Casino (1995)]"
15525,h1,523,"[Schindler's List (1993), Back to the Future (1985), Saving Private Ryan (1998), Shrek (2001), Toy Story (1995)]"
16624,h1,640,"[Rock, The (1996), Toy Story (1995), Twelve Monkeys (a.k.a. 12 Monkeys) (1995), Phenomenon (1996), Ransom (1996)]"
